# Vectorizacion de texto

# Funciones de limpieza

In [68]:
# Importo las librerias necesarias
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import string
from nltk.corpus import stopwords
import json
import glob
import re
from pprint import pprint
stops = stopwords.words("spanish")

In [69]:
# Funciones de limpieza de títulos

def remove_stops(text, stops):
    text = re.sub(r'(I{1,3}|IV|V|VI{1,3}|IX|X)[-.) ]+','',text)
    text = re.sub(r'[^\w\s -]','',text)
    text = re.sub(r'-',' ',text)
    text = text.lower()
    words = text.split()
    final = []
    for word in words:
        if word not in stops:
            final.append(word)
    final = " ".join(final)
    final = final.translate(str.maketrans("", "", string.punctuation))
    final = "".join([i for i in final if not i.isdigit()])
    while "  " in final:
        final = final.replace("  ", " ")
    return (final)

def clean_docs(docs):
    stops = stopwords.words("spanish")
    final = []
    for doc in docs:
        #clean_doc = remove_stops(doc, stops)
        final.append(doc)
    return (final)

# Creo las variables con titulos candidatos y encontrados.

In [70]:
# Creo las variables con titulos candidatos y encontrados.

encontrado = pd.read_csv("titulosEncontrados.csv", sep=',',  encoding='utf-8')
candidato = pd.read_csv("titulosCandidatos.csv", sep=',',  encoding='utf-8')
candidato = candidato.dropna()
encontrado = encontrado.dropna()
titEncontrado = clean_docs(encontrado['titulo'])
titCandidato = clean_docs(candidato['titulo'])

In [71]:
# Creo las variables que contienen los dataframes de titulos candidatos y encontrados.
dfTitCandidato = pd.read_csv('df_titulosCandidatos.csv', sep=',',  encoding='utf-8')
dfTitEncontrado = pd.read_csv('df_titulosEncontrados.csv', sep=',',  encoding='utf-8')
dfExpTitulosCandidatos = pd.read_csv('df_ExpTitulosCandidatos.csv', sep=',',  encoding='utf-8')

# CountVectorizer

In [72]:
# Importo las librerias necesarias.
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

'''
Creo el vectorizador - Función: countVectorizer.
Entreno el vectorizador con títulos candidatos a nivel de caracteres.
Creo las matrices de títulos candidatos y títulos encontrados.
'''
coun_vect = CountVectorizer(max_df=0.95,min_df=100,lowercase=True, analyzer="char", ngram_range=(1,3),max_features = 1000)
coun_vect.fit(titEncontrado)
count_candidatos = coun_vect.transform(titCandidato)
count_encontrados = coun_vect.transform(titEncontrado)
count_array = count_candidatos.toarray()
dfCountVec = pd.DataFrame(data=count_array,columns = coun_vect.get_feature_names_out())
dfCountVec

# Dimensiones: 
# encontrados 5286 rows × 300 columns
# candidatos 10105 rows × 300 columns

,,a,ac,an,au,c,cl,co,d,de,...,ut,uto,v,va,vi,x,xa,xam,z,za
0,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,0
3,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24241,6,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24242,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24243,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24244,7,1,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0


# Distancia cosenoidal

In [73]:
# Calculo de la similitud de vectores - Función: similitud coseno
from sklearn.metrics.pairwise import cosine_similarity
dist = cosine_similarity(count_candidatos, count_encontrados)

In [74]:
# Filas - Vectores - titulos candidatos
dist[:,0].shape

(24246,)

In [75]:
# Filas: titulos candidatos
# Columnas: titulos encontrados
dist.shape

(24246, 5179)

# BUSQUEDA DE TÍTULOS CANDIDATOS
## DISTANCIA COSENOIDAL

In [76]:
# Calculo la distancia entre títulos candidatos y títulos encontrados. Solo se guarda el valor mayor de similitud.
import numpy as np

candidatoList=[]
encotradoList=[]
distancia=[]
for i_cand in range(dist.shape[0]):
    i_enc = np.argmax(dist[i_cand])
    if dist[i_cand][i_enc] > 0.9:
        candidatoList.append(titCandidato[i_cand])
        encotradoList.append(titEncontrado[i_enc])
        distancia.append(dist[i_cand][i_enc])

# Creo un dataframe de Nuevos títulos encontrados y sus distancias cosenoidales.
dfNuevosCandidatosDistancia = pd.DataFrame({'candidato':candidatoList, 'encontrado': encotradoList, 'distancia': distancia})

In [77]:
dfNuevosCandidatosDistancia.to_csv('df_nuevosCandidatosDistancia.csv',index=False)
dfNuevosCandidatosDistancia.to_excel('df_nuevosCandidatosDistancia.xlsx',index=False)

# BUSQUEDA DE TÍTULOS CANDIDATOS
## Comparación de palabras

In [78]:
# Funcion para buscar titulos candidatos = titulos encontrados
def buscartitulo(tituloE, tituloC):
    titEncontrados = []
    for titulo in tituloE:
        for candidato in tituloC:
            if titulo == candidato:
                titEncontrados.append(candidato)
    return titEncontrados

    

In [79]:
# Creo los diccionarios de titulos candidatos y titulos encontrados.
encontradosTokens = encontrado.value_counts().head(200)
candidatoToken = candidato.value_counts().head(500)

In [80]:
encontradosTokens

titulo                                          
petitorio                                           461
conclusiones                                        386
consideraciones medico legales                      269
historia clinica                                    164
contestacion puntos pericia                         155
                                                   ... 
procedimientos utilizados para pericia                3
contestacion puntos pericia fijados sentenciante      3
antecedentes medico legales                           3
izquierdo                                             3
antecedentes hechos                                   3
Length: 200, dtype: int64

In [81]:
# Convierte a lista los titulos
def listartitulos(tokens):
    listTit=[]
    for i,j in tokens.items():
        i = remove_stops(str(i),stops)
        listTit.append(i)
    return listTit

In [82]:
# Convierto titulos a listas de titulos.
candidatosList = listartitulos(candidatoToken)
encontradosList = listartitulos(encontradosTokens)

In [83]:
# Creo una variable que contiene los titulos candidatos = titulos encontrados
# Diccionario de titulos candidatos para la busqueda.
candidatosEncontrados = buscartitulo(encontradosList, candidatosList)

In [84]:
candidatosEncontrados

['petitorio',
 'conclusiones',
 'consideraciones medico legales',
 'historia clinica',
 'contestacion puntos pericia',
 'antecedentes autos interes medico legal',
 'hechos',
 'conclusiones medico legales',
 'objeto',
 'consideraciones medicolegales',
 'datos personales',
 'examen actor',
 'examen fisico',
 'antecedentes autos',
 'bibliografia',
 'estudios complementarios',
 'antecedentes interes medico legal',
 'respuesta puntos pericia',
 'examen perito oficio',
 'puntos periciales',
 'puntos pericia',
 'incapacidad',
 'antecedentes',
 'identificacion actor',
 'conclusiones medicolegales',
 'columna cervical',
 'manifestacion ansiosa',
 'historia clinica actual',
 'historia clinica actual',
 'columna lumbosacra',
 'relato hechos',
 'antecedentes personales',
 'incapacidad actual',
 'actual',
 'conclusion',
 'examen medico',
 '',
 '',
 '',
 '',
 'tecnicas administradas',
 'presenta',
 'columna lumbar',
 'examenes complementarios',
 'diagnostico',
 'rodilla derecha',
 'antecedentes inte

# Búsqueda de títulos en los títulos candidatos

In [85]:
# Buscar_candidatos: compara títulos candidatos con el diccionario te títulos candidatos=encontrados
def buscar_candidatos(encontrado, candidato, ide, inicio, fin):
    indiceEncontrados=[]
    for i in range(len(candidato)):
        for e in encontrado:
            if candidato[i] == e:
                indiceEncontrados.append((ide[i],candidato[i], inicio[i], fin[i]))
    return indiceEncontrados

In [86]:
# Lista con los títulos candidatos que son títulos de sección, su índice y ubicación
indicesCandidatos = buscar_candidatos(candidatosEncontrados ,dfTitCandidato.titulo, dfTitCandidato.ide, dfTitCandidato.inicio, dfTitCandidato.fin)

In [87]:
# Creo un dataframe de nuevos titulos encontrados y su ubicacion.
dfUbicacionNuevosEncontrados = pd.DataFrame(indicesCandidatos, columns=['ide', 'titulo', 'inicio', 'fin'])

In [88]:
dfUbicacionNuevosEncontrados.to_csv('df_UbicacionNuevosEncontrados.csv',index=False)
dfUbicacionNuevosEncontrados.to_excel('df_UbicacionNuevosEncontrados.xlsx',index=False)

# Busqueda de expedientes sin titulos localizados

In [89]:
# Guardo en dfExpNuevosTitulos los expedientes y su indice dentro del dataframe limpio (con todos los expedientes)

dfExpSinTitulosEncontrados = dfExpTitulosCandidatos
for id_e in dfUbicacionNuevosEncontrados.ide:
    dfExpSinTitulosEncontrados = dfExpSinTitulosEncontrados.drop(dfExpSinTitulosEncontrados[dfExpSinTitulosEncontrados['ide'] ==id_e].index)
# dfExpNuevosTitulosEncontrados = pd.DataFrame(expConTitulosMayusculas, columns=['ide','expediente'])
# dfSinTitulosEncontrados = pd.DataFrame(expSinTitulosEncontrados, columns=['ide','expediente'])

# PRUEBAS

In [90]:
import seaborn as sns
import numpy as np

#sns.heatmap(dist)

In [91]:
import numpy
numpy.argmax(dist[0])



840

In [92]:
dist[0][437]

0.7046265129941619

In [93]:
print(titCandidato[0])
print(titEncontrado[437])

antecedentes medicos legales importancia
puntos pericia medica parte demandada galeno


# Count Pruebas

In [94]:
# Importo las librerias necesarias.
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

'''
Creo el vectorizador - Función: countVectorizer.
Entreno el vectorizador con títulos candidatos a nivel de caracteres.
Creo las matrices de títulos candidatos y títulos encontrados.
'''
coun_vect = CountVectorizer(max_df=0.99,min_df=1,lowercase=True, analyzer="char", ngram_range=(1,3),max_features = 1000)
coun_vect.fit(candidatosEncontrados)
count_candidatos = coun_vect.transform(titCandidato)
count_encontrados = coun_vect.transform(candidatosEncontrados)
count_array = count_candidatos.toarray()
dfCountVec = pd.DataFrame(data=count_array,columns = coun_vect.get_feature_names_out())
dfCountVec


,,a,ac,ad,an,au,c,ce,cl,co,...,ve,ven,vi,vic,x,xa,xam,z,zq,zqu
0,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24241,6,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24242,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24243,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24244,7,1,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0


In [95]:
# Calculo de la similitud de vectores - Función: similitud coseno
from sklearn.metrics.pairwise import cosine_similarity
dist = cosine_similarity(count_candidatos, count_encontrados)

In [96]:
dist.shape

(24246, 57)

In [97]:
# Calculo la distancia entre títulos candidatos y títulos encontrados. Solo se guarda el valor mayor de similitud.
from operator import index
import numpy as np

candidatoList=[]
encotradoList=[]
distancia=[]
for i_cand in range(dist.shape[0]):
    i_enc = np.argmax(dist[i_cand])
    if dist[i_cand][i_enc] > 0.9:
        candidatoList.append(titCandidato[i_cand])
        encotradoList.append(candidatosEncontrados[i_enc])
        distancia.append(dist[i_cand][i_enc])
    else:
        dfTitCandidato = dfTitCandidato.drop(dfTitCandidato[dfTitCandidato['titulo'] == titCandidato[i_cand]].index)
# Creo un dataframe de Nuevos títulos encontrados y sus distancias cosenoidales.
dfPruebaCandidatosDistancia = pd.DataFrame({'candidato':candidatoList, 'encontrado': encotradoList, 'distancia': distancia})

In [98]:
dfTitCandidato.shape

(3380, 6)

In [99]:
dfNuevo = dfPruebaCandidatosDistancia.drop_duplicates()

In [100]:
len(dfNuevo)

219